In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler

In [7]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [8]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [9]:
#Encoding categorical data

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])


In [10]:
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [11]:
#create dummy verialbe

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()


TypeError: __init__() got an unexpected keyword argument 'categorical_features'

TypeError: __init__() got an unexpected keyword argument 'categorical_features'.
Error means a function __init __ () does not give a desired parameter obtained, meaning that no parameters 'categorical_features'. The original version 0.22.1 scikit-learn later, OneHotEncoder no parameters categorical_features.
If your scikit-learn version still use parameters categorical_features forward in 0.22.1, can not go wrong, but there will be prompt Warning (red line), you can not specify the encoding sequence, prompting you to modify the ColumnTransformer, you can specify its parameters.

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
     

In [13]:
#ct = ColumnTransformer(transformers=[('Geograohy', OneHotEncoder(), [0,1])], remainder='passthrough')
#X = np.array(ct.fit_transform(X))

In [14]:
X.shape

(10000, 12)

In [15]:
# create dummy veriable trap means removing one column from it
X = X[:, 1:]
X

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

In [16]:
X.shape

(10000, 11)

In [17]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [18]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
np.shape(X_train)

(7500, 11)

In [20]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [21]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [22]:
classifier = KerasClassifier(build_fn = build_classifier)

In [23]:
parameters = {'batch_size': [5, 10],'epochs': [50, 100],'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 5)

In [24]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
6000/6000 [==============================] - 2s 373us/step - loss: 0.4661 - accuracy: 0.7985
Epoch 2/50
6000/6000 [==============================] - 2s 304us/step - loss: 0.4248 - accuracy: 0.7985
Epoch 3/50
6000/6000 [==============================] - 2s 279us/step - loss: 0.4191 - accuracy: 0.8067
Epoch 4/50
6000/6000 [==============================] - 2s 280us/step - loss: 0.4162 - accuracy: 0.8247
Epoch 5/50
6000/6000 [==============================] - 2s 279us/step - loss: 0.4141 - accuracy: 0.8310
Epoch 6/50
6000/6000 [==============================] - 2s 279us/step - loss: 0.4123 - accuracy: 0.8318
Epoch 7/50
6000/6000 [==============================] - 2s 282us/step - loss: 0.4113 - accuracy: 0.8318
Epoch 8/50
6000/6000 [==============================] - 2s 292us/step - loss: 0.4101 - accuracy: 0.8330
Epoch 9/50
6000/6000 [==============================] - 2s 292us/step - loss: 0.4097 - accuracy: 0.8328
Epoch 10/50
6000/6000 [==============================] - 2s 361u

6000/6000 [==============================] - 2s 370us/step - loss: 0.4039 - accuracy: 0.8335
Epoch 30/50
6000/6000 [==============================] - 2s 367us/step - loss: 0.4039 - accuracy: 0.8335
Epoch 31/50
6000/6000 [==============================] - 3s 444us/step - loss: 0.4036 - accuracy: 0.8332
Epoch 32/50
6000/6000 [==============================] - 2s 375us/step - loss: 0.4042 - accuracy: 0.8337
Epoch 33/50
6000/6000 [==============================] - 2s 373us/step - loss: 0.4035 - accuracy: 0.8328
Epoch 34/50
6000/6000 [==============================] - 2s 374us/step - loss: 0.4031 - accuracy: 0.8337
Epoch 35/50
6000/6000 [==============================] - 2s 375us/step - loss: 0.4031 - accuracy: 0.8337
Epoch 36/50
6000/6000 [==============================] - 2s 372us/step - loss: 0.4028 - accuracy: 0.8315
Epoch 37/50
6000/6000 [==============================] - 2s 369us/step - loss: 0.4031 - accuracy: 0.8325
Epoch 38/50
6000/6000 [==============================] - 3s 444us/s

6000/6000 [==============================] - 2s 345us/step - loss: 0.4089 - accuracy: 0.8328
Epoch 8/50
6000/6000 [==============================] - 2s 342us/step - loss: 0.4075 - accuracy: 0.8347
Epoch 9/50
6000/6000 [==============================] - 2s 339us/step - loss: 0.4064 - accuracy: 0.8342
Epoch 10/50
6000/6000 [==============================] - 2s 341us/step - loss: 0.4057 - accuracy: 0.8353
Epoch 11/50
6000/6000 [==============================] - 2s 344us/step - loss: 0.4045 - accuracy: 0.8350
Epoch 12/50
6000/6000 [==============================] - 2s 350us/step - loss: 0.4042 - accuracy: 0.8350
Epoch 13/50
6000/6000 [==============================] - 2s 415us/step - loss: 0.4030 - accuracy: 0.8358
Epoch 14/50
6000/6000 [==============================] - 2s 345us/step - loss: 0.4022 - accuracy: 0.8350
Epoch 15/50
6000/6000 [==============================] - 2s 349us/step - loss: 0.4023 - accuracy: 0.8372
Epoch 16/50
6000/6000 [==============================] - 2s 354us/ste

6000/6000 [==============================] - 2s 407us/step - loss: 0.3476 - accuracy: 0.8610
Epoch 36/50
6000/6000 [==============================] - 2s 337us/step - loss: 0.3469 - accuracy: 0.8632
Epoch 37/50
6000/6000 [==============================] - 2s 335us/step - loss: 0.3478 - accuracy: 0.8592
Epoch 38/50
6000/6000 [==============================] - 2s 334us/step - loss: 0.3478 - accuracy: 0.8595
Epoch 39/50
6000/6000 [==============================] - 2s 340us/step - loss: 0.3466 - accuracy: 0.8607
Epoch 40/50
6000/6000 [==============================] - 2s 333us/step - loss: 0.3470 - accuracy: 0.8607
Epoch 41/50
6000/6000 [==============================] - 2s 336us/step - loss: 0.3450 - accuracy: 0.8625
Epoch 42/50
6000/6000 [==============================] - 2s 336us/step - loss: 0.3467 - accuracy: 0.8608
Epoch 43/50
6000/6000 [==============================] - 2s 403us/step - loss: 0.3464 - accuracy: 0.8610
Epoch 44/50
6000/6000 [==============================] - 2s 337us/s

6000/6000 [==============================] - 2s 327us/step - loss: 0.4099 - accuracy: 0.8347
Epoch 14/50
6000/6000 [==============================] - 2s 340us/step - loss: 0.4091 - accuracy: 0.8347
Epoch 15/50
6000/6000 [==============================] - 2s 300us/step - loss: 0.4088 - accuracy: 0.8352
Epoch 16/50
6000/6000 [==============================] - 2s 298us/step - loss: 0.4092 - accuracy: 0.8353
Epoch 17/50
6000/6000 [==============================] - 2s 300us/step - loss: 0.4080 - accuracy: 0.8328
Epoch 18/50
6000/6000 [==============================] - 2s 300us/step - loss: 0.4076 - accuracy: 0.8330
Epoch 19/50
6000/6000 [==============================] - 2s 297us/step - loss: 0.4080 - accuracy: 0.8348
Epoch 20/50
6000/6000 [==============================] - 2s 297us/step - loss: 0.4070 - accuracy: 0.8338
Epoch 21/50
6000/6000 [==============================] - 2s 296us/step - loss: 0.4070 - accuracy: 0.8345
Epoch 22/50
6000/6000 [==============================] - 2s 368us/s

...

6000/6000 [==============================] - 1s 171us/step - loss: 0.3467 - accuracy: 0.8602
Epoch 84/100
6000/6000 [==============================] - 1s 174us/step - loss: 0.3474 - accuracy: 0.8588
Epoch 85/100
6000/6000 [==============================] - 1s 173us/step - loss: 0.3467 - accuracy: 0.8612
Epoch 86/100
6000/6000 [==============================] - 1s 175us/step - loss: 0.3465 - accuracy: 0.8605
Epoch 87/100
6000/6000 [==============================] - 1s 176us/step - loss: 0.3472 - accuracy: 0.8587
Epoch 88/100
6000/6000 [==============================] - 1s 172us/step - loss: 0.3463 - accuracy: 0.8605
Epoch 89/100
6000/6000 [==============================] - 1s 172us/step - loss: 0.3469 - accuracy: 0.8592
Epoch 90/100
6000/6000 [==============================] - 1s 247us/step - loss: 0.3459 - accuracy: 0.8593
Epoch 91/100
6000/6000 [==============================] - 1s 180us/step - loss: 0.3458 - accuracy: 0.8607
Epoch 92/100
6000/6000 [==============================] - E

6000/6000 [==============================] - 1s 177us/step - loss: 0.3980 - accuracy: 0.8357
Epoch 61/100
6000/6000 [==============================] - 1s 178us/step - loss: 0.3981 - accuracy: 0.8380
Epoch 62/100
6000/6000 [==============================] - 2s 257us/step - loss: 0.3980 - accuracy: 0.8377
Epoch 63/100
6000/6000 [==============================] - 1s 181us/step - loss: 0.3981 - accuracy: 0.8370
Epoch 64/100
6000/6000 [==============================] - 1s 180us/step - loss: 0.3984 - accuracy: 0.8373
Epoch 65/100
6000/6000 [==============================] - 1s 179us/step - loss: 0.3970 - accuracy: 0.8380
Epoch 66/100
6000/6000 [==============================] - 1s 180us/step - loss: 0.3983 - accuracy: 0.8353
Epoch 67/100
6000/6000 [==============================] - 1s 180us/step - loss: 0.3973 - accuracy: 0.8382
Epoch 68/100
6000/6000 [==============================] - 1s 181us/step - loss: 0.3973 - accuracy: 0.8372
Epoch 69/100
6000/6000 [==============================] - 1

6000/6000 [==============================] - 1s 203us/step - loss: 0.4034 - accuracy: 0.8342
Epoch 38/100
6000/6000 [==============================] - 1s 194us/step - loss: 0.4039 - accuracy: 0.8348
Epoch 39/100
6000/6000 [==============================] - 1s 209us/step - loss: 0.4036 - accuracy: 0.8342
Epoch 40/100
6000/6000 [==============================] - 1s 197us/step - loss: 0.4029 - accuracy: 0.8360
Epoch 41/100
6000/6000 [==============================] - 1s 199us/step - loss: 0.4034 - accuracy: 0.8332
Epoch 42/100
6000/6000 [==============================] - 1s 198us/step - loss: 0.4029 - accuracy: 0.8337
Epoch 43/100
6000/6000 [==============================] - 1s 200us/step - loss: 0.4031 - accuracy: 0.8368
Epoch 44/100
6000/6000 [==============================] - 1s 200us/step - loss: 0.4032 - accuracy: 0.8335
Epoch 45/100
6000/6000 [==============================] - 1s 202us/step - loss: 0.4030 - accuracy: 0.8352
Epoch 46/100
6000/6000 [==============================] - 1

7500/7500 [==============================] - 3s 401us/step - loss: 0.4061 - accuracy: 0.8339
Epoch 15/100
7500/7500 [==============================] - 3s 407us/step - loss: 0.4055 - accuracy: 0.8333
Epoch 16/100
7500/7500 [==============================] - 3s 397us/step - loss: 0.4054 - accuracy: 0.8351
Epoch 17/100
7500/7500 [==============================] - 3s 399us/step - loss: 0.4048 - accuracy: 0.8352
Epoch 18/100
7500/7500 [==============================] - 4s 474us/step - loss: 0.4045 - accuracy: 0.8340
Epoch 19/100
7500/7500 [==============================] - 3s 407us/step - loss: 0.4044 - accuracy: 0.8343
Epoch 20/100
7500/7500 [==============================] - 3s 411us/step - loss: 0.4034 - accuracy: 0.8324
Epoch 21/100
7500/7500 [==============================] - 3s 406us/step - loss: 0.4041 - accuracy: 0.8344
Epoch 22/100
7500/7500 [==============================] - 3s 403us/step - loss: 0.4037 - accuracy: 0.8336
Epoch 23/100
7500/7500 [==============================] - 4

7500/7500 [==============================] - 4s 506us/step - loss: 0.4009 - accuracy: 0.8356
Epoch 92/100
7500/7500 [==============================] - 3s 415us/step - loss: 0.4003 - accuracy: 0.8360
Epoch 93/100
7500/7500 [==============================] - 3s 412us/step - loss: 0.4006 - accuracy: 0.8339
Epoch 94/100
7500/7500 [==============================] - 3s 408us/step - loss: 0.4004 - accuracy: 0.8352
Epoch 95/100
7500/7500 [==============================] - 3s 403us/step - loss: 0.4003 - accuracy: 0.8347
Epoch 96/100
7500/7500 [==============================] - 4s 471us/step - loss: 0.4006 - accuracy: 0.8337
Epoch 97/100
7500/7500 [==============================] - 3s 440us/step - loss: 0.4000 - accuracy: 0.8345
Epoch 98/100
7500/7500 [==============================] - 4s 472us/step - loss: 0.4002 - accuracy: 0.8348
Epoch 99/100
7500/7500 [==============================] - 3s 390us/step - loss: 0.4005 - accuracy: 0.8359
Epoch 100/100
7500/7500 [==============================] - 

In [29]:
best_parameters = grid_search.best_params_
best_parameters

{'batch_size': 5, 'epochs': 100, 'optimizer': 'adam'}

In [30]:
best_accuracy = grid_search.best_score_
best_accuracy

0.8502666666666666